# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298532292394                   -0.85    4.0
  2   -8.300222899980       -2.77       -1.25    1.0
  3   -8.300447751154       -3.65       -1.89    3.4
  4   -8.300462184578       -4.84       -2.75    3.0
  5   -8.300463902703       -5.76       -3.04    8.0
  6   -8.300464265637       -6.44       -3.23    1.0
  7   -8.300464449406       -6.74       -3.36    6.1
  8   -8.300464552615       -6.99       -3.50    1.0
  9   -8.300464622370       -7.16       -3.71    1.2
 10   -8.300464638150       -7.80       -3.92    1.0
 11   -8.300464643503       -8.27       -4.28    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64208161693                   -0.70    5.5
  2   -16.67855579465       -1.44       -1.14    1.6
  3   -16.67921975427       -3.18       -1.87    2.9
  4   -16.67928100700       -4.21       -2.76    5.0
  5   -16.67928604618       -5.30       -3.10    4.9
  6   -16.67928621750       -6.77       -3.53    1.1
  7   -16.67928622203       -8.34       -4.03    1.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32551547915                   -0.56    6.4
  2   -33.33273998329       -2.14       -1.00    1.1
  3   -33.33436137445       -2.79       -1.71    3.0
  4   -33.33607820842       -2.77       -2.49    3.2
  5   -33.33690591382       -3.08       -2.51    7.6
  6   -33.33693009794       -4.62       -2.74    8.4
  7   -33.33694386945       -4.86       -3.58    2.8
  8   -33.33694382062   +   -7.31       -3.74    5.0
  9   -33.33694392376       -6.99       -4.39    3.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298411118154                   -0.85    4.6
  2   -8.300258070643       -2.73       -1.59    1.0
  3   -8.300423065106       -3.78       -2.39    7.6
  4   -8.300316735360   +   -3.97       -2.17   17.2
  5   -8.300464594025       -3.83       -3.84    3.0
  6   -8.300464608793       -7.83       -4.01    5.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32563523928                   -0.56    7.1
  2   -33.29672812047   +   -1.54       -1.26    1.1
  3   +2.351405257543   +    1.55       -0.29    8.8
  4   -33.31400966895        1.55       -1.61    7.6
  5   -33.20624782282   +   -0.97       -1.32    3.9
  6   -31.08849929583   +    0.33       -0.89    4.4
  7   -33.29603859029        0.34       -1.72    7.2
  8   -33.33517579725       -1.41       -2.25    3.1
  9   -33.33502295225   +   -3.82       -2.09    2.8
 10   -33.33684667036       -2.74       -2.83    2.2
 11   -33.33689957319       -4.28       -2.96    1.8
 12   -33.33692212455       -4.65       -3.09    2.1
 13   -33.33694198459       -4.70       -3.50    2.8
 14   -33.33694326737       -5.89       -3.95    3.5
 15   -33.33694364156       -6.43       -4.25    3.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.